In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data=pd.read_csv('../input/how-much-did-it-rain-ii/train.zip')

In [ ]:
train_data.dropna(axis=0, subset=['Expected'], inplace=True)
train_d = train_data[train_data['Expected'] < 73].copy()
del train_data


In [ ]:
train_d.shape

In [ ]:
train_d.head()

In [ ]:
row=float(len(train_d))
col_rem=[col for col in train_d.columns if col!='Expected' and train_d[col].isnull().sum()>int(0.4*row) ]

In [ ]:
train_d.drop(col_rem,axis=1,inplace=True)

In [ ]:
del row

In [ ]:
train_d.shape

In [ ]:
train_d.isnull().sum()

In [ ]:
from sklearn.impute import SimpleImputer
my_imputer=SimpleImputer(strategy='median')
imputed_train = pd.DataFrame(my_imputer.fit_transform(train_d))
imputed_train.columns = train_d.columns


In [ ]:
imputed_train.isnull().any()

In [ ]:
imputed_train.head()

In [ ]:
imputed_train['Id'].astype(int)

In [ ]:
train_group=imputed_train.groupby('Id',as_index=False)[['minutes_past','radardist_km','RefComposite_5x5_90th','Expected']].median()

In [ ]:
train_group.head()

In [ ]:
y=train_group['Expected']
train_unnorm=train_group.loc[:,['minutes_past','radardist_km','RefComposite_5x5_90th']]
                  

In [ ]:
train_unnorm.head()

In [ ]:

from sklearn import preprocessing

x = train_unnorm.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
final_train = pd.DataFrame(x_scaled)


In [ ]:
final_train.head()

In [ ]:
final_train.columns=train_unnorm.columns

In [ ]:
final_train.head()

In [ ]:
final_train_arr=final_train.to_numpy()

In [ ]:
final_train_arr[0:6]

In [ ]:
y_arr=y.to_numpy()

In [ ]:
y_arr.shape

In [ ]:
y_arr[0:6]

In [ ]:
f_arr=final_train_arr.reshape(len(final_train_arr),1,3)

In [ ]:
f_arr.shape

In [ ]:
f_arr[0:6]

In [ ]:
# detect and init the TPU
import tensorflow as tf
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)


# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
del x


In [ ]:
y_a=y_arr.reshape(len(final_train),1,1)

In [ ]:
y_a.shape

In [ ]:
with tpu_strategy.scope():
    from tensorflow.keras.models import Sequential
    from tensorflow.python.keras.layers import LSTM
    model1 =Sequential()
    model1.add(LSTM(1, input_shape=(1,3),return_sequences= True))
    model1.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model1.fit(f_arr, y_a, epochs=20,batch_size=1674)
    



In [ ]:
del imputed_train
del train_group
del train_d


In [ ]:
del test_data

In [ ]:
test_data=pd.read_csv('../input/how-much-did-it-rain-ii/test.zip')

In [ ]:
test_data.head()

In [ ]:
test_data.drop(col_rem,axis=1,inplace=True)

In [ ]:
test_data.head()

In [ ]:
test_data.isnull().any()

In [ ]:
test_data.fillna(test_data['RefComposite_5x5_90th'].median(), inplace=True)

In [ ]:
test_data.isnull().any()

In [ ]:
test_group=test_data.groupby('Id',as_index=False)[['minutes_past','radardist_km','RefComposite_5x5_90th']].median()

In [ ]:
test_group.head()

In [ ]:
test_group.drop('Id',axis=1,inplace=True)

In [ ]:
test_group.head()

In [ ]:
x = test_group.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
final_test = pd.DataFrame(x_scaled)

In [ ]:
final_test.head()

In [ ]:
fin_arr=final_test.to_numpy()
f_a=fin_arr.reshape(len(final_test),1,3)

In [ ]:
f_a.shape

In [ ]:
preds=model1.predict(f_a)

In [ ]:
preds.shape

In [ ]:
new_preds=preds.flatten()

In [ ]:
new_preds.shape

In [ ]:
len(final_test)

In [ ]:
preds_fin=new_preds[0:len(final_test)]

In [ ]:
len(preds_fin)

In [ ]:
output = pd.DataFrame({'Id': final_test.index+1,
                       'Expected': preds_fin})
output.head()
#output.to_csv('submission.csv', index=False)

In [ ]:
output.to_csv('sub3.csv', index=False)